In [1]:
# randomNG = 'Philox'
# randomNG = 'PCG64DXSM'
# randomNG = 'MT19937'
randomNG = 'SFC64'
import importlib

In [2]:
import numpy as np
import time
from IPython.display import clear_output
import datetime
import numba
import concurrent.futures
import math
import numpy.random
bitNG = getattr(numpy.random, randomNG)
from numpy.random import Generator, SeedSequence

from stdp_FNs_004first_NaNcorrectionWITHmaxCAP import stdp_epoch, initenv

In [3]:
# initial parameters

# lets try to get a full play going. repeating all of the constants here
runs = 20
rgs = [Generator(bitNG(s)) for s in SeedSequence().spawn(runs)]
rng = rgs[0]


spikemem = 120
epochlen = 600
epochmem = 6*epochlen

# total number of timesteps that will be simulated
timesteps = epochlen*5000
current_tstep = 0 #whatever the current timestep is

# number of I neurons
inum = 7*7-1
stimsize = 7 #this is just so I can pass posenv2stim a variable rather than hard coding
# number of H neurons
hnum = 28*28
# number of O neurons
onum = 3*3 -1

Tc = 80 # a constant used in the tr_k computation, paper says 40ms but each timestep is 0.5ms
K = 0.04 # a constant used in the tr_k computation
Srp_basic = -0.00001
Srp_reward = 0.4
Srp_punish = -0.001
c_deltak = 1
alpha = .0365
beta_e = 0.133
mu = 0.0005
sigma = 0.06
ifirefactor = 3 #how much I^ext does a particle in the environment generate
hfirefactor = 0.1 #can use this to amplify the amount of current that h layer neurons receive if neccessary
ofirefactor = 1

np.set_printoptions(linewidth=190, threshold=60*60) #allows nummpy to print entire environment display

In [4]:
# initializing variables
# note: will likely turn this and the next cell into a single function once confirmed that everything is functioning properly


ispikes = np.zeros((spikemem, inum))
hspikes = np.zeros((spikemem, hnum))
ospikes = np.zeros((spikemem, onum))

# sum traces for synapses between h and o
epoch_hotrks = np.zeros((epochmem, hnum, onum))
epoch_ohtrks = np.zeros((epochmem, hnum, onum))

outspikecount = np.zeros(onum)

# an array storing Wij's for synaptic weights between I and H
wih = rng.random((inum, hnum))
# an array storing Wij's for synaptic weights between H and O
who = rng.random((hnum, onum))
# an array storing Wij's for synaptic weights between H and O INHIBITORY
whoI = rng.random((hnum, onum))

environment = np.zeros((50,50), dtype = int)
environment[25][25] = 1 #this makes sure no food or punishment gets place on starting location
envirnoment = initenv(environment, rng)
environment[25][25] = 0 # removes above 1 that gauranteed no food or punishment got placed on starting location
position = np.array([25,25], dtype = int) #since enviornment is randomly generated for each run, it's okay to always start the network in the same place


# an array storing Wij's for synaptic weights between I and H
wih = rng.random((inum, hnum))
ihtargets = rng.random((inum, hnum))
# an array storing Wij's for synaptic weights between H and O
who = rng.random((hnum, onum))
hotargets = rng.random((hnum, onum))
# an array storing Wij's for synaptic weights between H and O INHIBITORY
whoI = rng.random((hnum, onum))
hoItargets = rng.random((hnum, onum))

# target sum of weights for each o nuron to use in weight normalization
eDtargets = np.sum(who, axis=0)
iDtargets = np.sum(whoI, axis=0)
hdnomDtar = 1/hnum

# membrane potentials:
vi = np.ones(inum)*-1
vh = np.ones(hnum)*-1
vo = np.ones(onum)*-1

# currents for the neurons 
icur = np.zeros(inum)
hcur = np.zeros(hnum)
ocur = np.zeros(onum)


epochSrp = -0.0001 #initial value for srp that will be updated 


In [5]:
# now let's actually do the full run

for current_tstep in range(0, timesteps, epochlen):
    
    #random values for each epoch
    randfood = rng.integers(50, size=(20, 2))
    randmove = rng.integers(onum)
    randchance = rng.random()
    
    wih, who, whoI, ispikes, hspikes, ospikes, epoch_hotrks, epoch_ohtrks, output_spikes, position, epochSrp, \
    environment = stdp_epoch(epochlen, current_tstep, epochmem, position, environment, stimsize, inum, hnum, onum, vi, vo, vh, icur, hcur, ocur, 
                  beta_e, alpha, K, mu, Tc, sigma, spikemem, ispikes, hspikes, ospikes, ifirefactor, hfirefactor, ofirefactor, eDtargets, iDtargets,
                  wih, who, whoI, hotargets, hoItargets, epochSrp, epoch_hotrks, epoch_ohtrks, randfood, Srp_basic, Srp_reward, Srp_punish, 
                  randmove, randchance, c_deltak)
    
    clear_output(wait=True)
#     lets display the environment with position
    environmentdisplay = envirnoment.copy()
    environmentdisplay[position[0]][position[1]] = 2
    print(environmentdisplay)
    print(position)
    print(output_spikes)

pass_abc = 5001
np.save(f'nlg{pass_abc}wih.npy', wih)
np.save(f'nlg{pass_abc}who.npy', who)
np.save(f'nlg{pass_abc}whoI.npy', whoI)

[[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0

In [5]:
pass_abc = 5001
timesteps = epochlen*1000

wih = np.load(f'nlg{pass_abc}wih.npy')
who = np.load(f'nlg{pass_abc}who.npy')
whoI = np.load(f'nlg{pass_abc}whoI.npy')
print(wih)
print(who)
print(whoI)

[[7.32828889e-02 1.00000110e-07 1.00000290e-07 ... 1.60860843e-01 7.30844290e-02 6.12145718e-02]
 [1.49856127e-01 8.45075665e-02 1.00000197e-07 ... 5.03033151e-02 1.00949279e-01 1.00000028e-07]
 [9.96545747e-02 5.84320895e-02 2.70293428e-01 ... 1.02506327e-01 8.05008214e-02 1.00000048e-07]
 ...
 [3.36176142e-01 1.71091226e-01 1.00000223e-07 ... 1.00000131e-07 1.34811580e-01 1.13655090e-01]
 [1.36542973e-01 2.62201638e-01 1.00000163e-07 ... 1.21274198e-01 7.14012379e-02 8.44953944e-03]
 [1.02497253e-01 1.00000138e-07 6.40867313e-02 ... 1.03402395e-01 1.89295307e-01 1.35646752e-01]]
[[0.54794525 0.53507718 0.64353966 ... 0.84366739 0.51835216 0.45179771]
 [0.58990106 0.58490467 0.65510791 ... 0.71636748 0.56261894 0.70903216]
 [0.62515067 0.62350249 0.57730951 ... 0.91467263 0.61777867 0.75981912]
 ...
 [0.54320707 0.55893466 0.52135518 ... 0.55450486 0.49754166 0.54917527]
 [0.48825601 0.29073355 0.12110387 ... 0.06978943 0.00171863 0.00193388]
 [0.54268747 0.52542754 0.60140078 ... 0.6

In [6]:




for current_tstep in range(0, timesteps, epochlen):
    
    #random values for each epoch
    randfood = rng.integers(50, size=(20, 2))
    randmove = rng.integers(onum)
    randchance = rng.random()
    
    wih, who, whoI, ispikes, hspikes, ospikes, epoch_hotrks, epoch_ohtrks, output_spikes, position, epochSrp, \
    environment = stdp_epoch(epochlen, current_tstep, epochmem, position, environment, stimsize, inum, hnum, onum, vi, vo, vh, icur, hcur, ocur, 
                  beta_e, alpha, K, mu, Tc, sigma, spikemem, ispikes, hspikes, ospikes, ifirefactor, hfirefactor, ofirefactor, eDtargets, iDtargets,
                  wih, who, whoI, hotargets, hoItargets, epochSrp, epoch_hotrks, epoch_ohtrks, randfood, Srp_basic, Srp_reward, Srp_punish, 
                  randmove, randchance, c_deltak)
    
    clear_output(wait=True)
#     lets display the environment with position
    environmentdisplay = envirnoment.copy()
    environmentdisplay[position[0]][position[1]] = 2
    print(environmentdisplay)
    print(position)
    print(output_spikes)

pass_abc = 5001
np.save(f'nlg{pass_abc}Bwih.npy', wih)
np.save(f'nlg{pass_abc}Bwho.npy', who)
np.save(f'nlg{pass_abc}BwhoI.npy', whoI)


[[0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 1 1 0 0 1 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0